The first thing we do is install all the libraries that we are going to need to be able to make the connection to the database, extract that data, and be able to clean it, in addition to importing the 'establish_connection' function which is what It allows us, as its name says, to make the connection with our database located in PostgreSQL

In [2]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from db_conexion import establecer_conexion

We proceed to define some variables, which will make it possible for us to extract the data from the database that we have, in this case, the variable conn, what it does is extract the database. With the cursor variable, what allows us to go through the data in our database, and finally, we create a query that allows us to create an SQL function to be able to extract the entire table, and then save it in a DataFrame and be able to do everything the possible analysis and visualization.

In [7]:
conn, cursor = establecer_conexion()

# Consulta SQL para seleccionar los datos
query = "SELECT * FROM us_accidents"

# Leer los datos en un DataFrame de pandas
df = pd.read_sql_query(query, conn)

Conexion exitosa a la base de datos


C:\Users\USUARIO\AppData\Local\Temp\ipykernel_14760\4066787522.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


Here what we simply do is verify the database, to see if the connection with the database was made correctly.

In [ ]:
import pandas as pd

# Configurar pandas para mostrar más filas y columnas
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)


# Mostrar las primeras 20 filas en formato tabular
df.head(20)

Then we proceed with all the cleaning.

1. First of all, we chose the columns that we saw pertinent to eliminate, which we eliminated because they did not give us relevant information, or had most of their data null, or simply there were more columns with the same information with which we could guide ourselves.

2. For meteorological variables, such as temperature or wind speed, what we did was a numerical estimate taking into account the existing values ​​in this column, then we could use the mode, the mean, the most frequent value or the closest value

3. Then we proceed to eliminate rows with null values, which makes it even easier for us to read the existing data in the column, and then proceed to count both the number of null values ​​and the number of empty values, in order to do so. finally being able to have a cleaner and more readable database.

In [ ]:
import pandas as pd

# Columnas a eliminar
columns_to_drop = ['id', 'source', 'country', 'description', 'end_lat', 'end_lng', 
                'civil_twilight', 'nautical_twilight', 'astronomical_twilight']

# Eliminación de las columnas
df_cleaned = df.drop(columns=columns_to_drop)

# Imputar valores nulos en columnas numéricas con la media
df_cleaned['temperature_f'].fillna(df_cleaned['temperature_f'].mean(), inplace=True)

# Imputar valores nulos en columnas categóricas con la moda
df_cleaned['weather_condition'].fillna(df_cleaned['weather_condition'].mode()[0], inplace=True)

# Imputar columnas numéricas con la media
num_cols = ['wind_chill_f', 'humidity_percent', 'pressure_in', 'visibility_mi', 'wind_speed_mph', 'precipitation_in']
df_cleaned[num_cols] = df_cleaned[num_cols].apply(lambda col: col.fillna(col.mean()))

# Imputar columna categórica con el valor más frecuente
df_cleaned['wind_direction'] = df_cleaned['wind_direction'].fillna(df_cleaned['wind_direction'].mode()[0])

# Imputar Weather_Timestamp con el valor más cercano (por tiempo)
df_cleaned['weather_timestamp'] = df_cleaned['weather_timestamp'].fillna(method='ffill')

# Eliminar filas que contienen valores nulos
df_cleaned.dropna(inplace=True)

# Contar valores nulos (NaN) en cada columna
nan_counts = df_cleaned.isna().sum()

# Contar valores vacíos ('') en cada columna
empty_counts = (df_cleaned == '').sum()

# Combinar los conteos en un solo DataFrame para visualizar mejor
null_summary = pd.DataFrame({
    'NaN Count': nan_counts,
    'Empty String Count': empty_counts,
    'Total Missing': nan_counts + empty_counts
})

# Mostrar el resumen
print(null_summary)

# Configuración para mostrar más filas y columnas si es necesario
pd.set_option('display.max_rows', 10000)
pd.set_option('display.max_columns', None)

# Mostrar el DataFrame en formato tabular
df_cleaned.head(100)
